In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from string import punctuation
from scipy.sparse import vstack, hstack, csr_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
import lightgbm as lgb
import time

In [35]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [36]:
#####   removing punctuation    ######
import string
def remove_punctuation(sentence: str) -> str:
    return sentence.translate(str.maketrans('', '', string.punctuation))
print(remove_punctuation("I,am!a girl."))

Iama girl


In [37]:
########## removing digits     #########
def remove_digits(x):
    x = ''.join([i for i in x if not i.isdigit()])
    return x
print(remove_digits("hey1 boy! 123"))

hey boy! 


In [38]:
############ installing stopwords     ############
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Anurag\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [39]:
########## removing stopwords    #########
from nltk.corpus import stopwords
stop = stopwords.words('english')
def remove_stop_words(x):
    x = ' '.join([i for i in x.lower().split(' ') if i not in stop])
    return x
#print(stop)
print(remove_stop_words("i me myself am you're good"))

good


In [40]:
######## converting to lower case    #####
def to_lower(x):
    return x.lower()
print(to_lower("MANSI IS A GOOD GIRL"))

mansi is a good girl


In [41]:
# handle_missing_values - Basic data imputation of missing values
def handle_missing_values(df):
    df['dominant_color'].fillna(value='missing', inplace=True)
    df['dominant_material'].fillna(value='None', inplace=True)
    df['product_type'].fillna(value='None', inplace=True)

In [42]:
# to_categorical - Converts Categorical Features 
def to_categorical(df):
    df['brand'] = df['brand'].astype('category')
    df['category'] = df['category'].astype('category')
    #df['item_condition_id'] = df['item_condition_id'].astype('category')

In [10]:
####### changing in_stock as shipping and others     ######
#text = open("C:/Users/Anurag/Desktop/myntradata.csv", "r")
#text = ''.join([i for i in text]) \
 #   .replace("In Stock", "0") \
  #  .replace('Out of Stock','1')
#x = open("C:/Users/Anurag/Desktop/final.csv","w")
#x.writelines(text)
#x.close()
#read input file
#import pandas as pd
#import re
#K=3
#splt_char = "/"
#df = pd.read_csv("C:/Users/Anurag/Desktop/myntradata.csv")
#df.head()
#for i in range(len(df)):
  #temp = df.loc[i,'category'].split(splt_char)
  #res = splt_char.join(temp[:K]), splt_char.join(temp[K:])
  #df.loc[i, 'category'] = res[0]
#df.to_csv("C:/Users/Anurag/Desktop/myntradata.csv", index=False)

In [11]:
#########################################################################################################################################################
#########################################################################################################################################################
#########################################################################################################################################################
################################################                     MY ML PART                  ########################################################
#########################################################################################################################################################
#########################################################################################################################################################
#########################################################################################################################################################

In [43]:

train1= pd.read_csv('C:/Users/Anurag/Desktop/MLMINI/final.csv')
train1.drop(['images','complete_the_look'],axis=1,inplace=True)
train1.head()
#len(train.columns)

################# for changing category   #################

splt_char = "/"
K=3
for i in range(len(train1)):
    s=str(train1.loc[i, 'category'])
    temp =s.split(splt_char)
    res = splt_char.join(temp[:K]), splt_char.join(temp[K:])
    train1.loc[i, 'category'] = res[0]
train1.head()

,product_id,size,brand,dominant_material,name,dominant_color,product_type,product_details,category,price,shipping
0,6937673,XL,IMARA,Polyester,IMARA Women Black Solid Top,Black,Top,"Black solid woven regular top,has a V-neck, th...",Clothing/Women/Tops,959,1
1,7441182,XL,House of Pataudi,cotton,House of Pataudi Men Black Printed Straight Kurta,Black,Straight Kurta,"Black printed straight kurta, has a mandarin c...",Clothing/Men/Kurtas,799,0
2,9245141,S,The White Willow,Cotton,The White Willow Unisex Off-White Therapedic M...,White,NaN,Set content: 1 pillow Colour: Off white Fill...,Home/Unisex/Pillows,1399,0
3,1314889,Onesize,Dupatta Bazaar,Chiffon,Dupatta Bazaar Pink Dupatta,Pink,Dupatta,Pink dupatta&nbsp;with crinkled effect,Clothing/Women/Dupatta,349,0
4,7705322,S,Manyavar,Silk,Manyavar Men Yellow & White Self Design Kurta ...,Yellow,Kurta with Churidar,Yellow and white self design kurta with churid...,Clothing/Men/Kurta Sets,2999,1


In [13]:
#########################################################################################################################################################
#########################################################################################################################################################
#########################################################################################################################################################
##########################################                     CATEGORIES                    ############################################################
#########################################################################################################################################################
#########################################################################################################################################################
#########################################################################################################################################################

In [44]:
#############  category wise distribution  ###########

def transform_category_name(category):
    try:
        main, sub1, sub2= category.split('/')
        return main, sub1, sub2
    except:
        return np.nan, np.nan, np.nan

train1['category'], train1['category_sub1'], train1['category_sub2'] = zip(*train1['category'].apply(transform_category_name))
cat_train = train1[['category','category_sub1','category_sub2', 'price']]
cat_train.head()

,category,category_sub1,category_sub2,price
0,Clothing,Women,Tops,959
1,Clothing,Men,Kurtas,799
2,Home,Unisex,Pillows,1399
3,Clothing,Women,Dupatta,349
4,Clothing,Men,Kurta Sets,2999


In [45]:
################        I am on 3d

train1.product_details = train1.product_details.astype(str)

descr = train1[['product_details', 'price']]
descr['count'] = descr['product_details'].apply(lambda x : len(str(x)))

descr['product_details'] = descr['product_details'].apply(remove_digits)
descr['product_details'] = descr['product_details'].apply(remove_punctuation)
descr['product_details'] = descr['product_details'].apply(remove_stop_words)

descr.head(5)

<ipython-input-45-4f4857f0d0c8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  descr['count'] = descr['product_details'].apply(lambda x : len(str(x)))
<ipython-input-45-4f4857f0d0c8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  descr['product_details'] = descr['product_details'].apply(remove_digits)
<ipython-input-45-4f4857f0d0c8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

,product_details,price,count
0,black solid woven regular tophas vneck threequ...,959,85
1,black printed straight kurta mandarin collar l...,799,107
2,set content pillow colour white filling mem...,1399,242
3,pink dupattanbspwith crinkled effect,349,38
4,yellow white self design kurta churidaryellow ...,2999,185


In [46]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
descr['product_details'] = descr['product_details'].apply(porter.stem)
descr.head(10)

<ipython-input-46-10f86e2ca5b1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  descr['product_details'] = descr['product_details'].apply(porter.stem)


,product_details,price,count
0,black solid woven regular tophas vneck threequ...,959,85
1,black printed straight kurta mandarin collar l...,799,107
2,set content pillow colour white filling mem...,1399,242
3,pink dupattanbspwith crinkled effect,349,38
4,yellow white self design kurta churidaryellow ...,2999,185
5,beige solid kurta pyjamas beige straight knee ...,979,165
6,green coloured freesize dabu print cotton skir...,1572,153
7,blue white printed aline kurta mandarin collar...,479,124
8,maroon solid aline kurta round neck threequart...,1359,78
9,orange white printed kurta pyjamas orange stra...,586,183


In [47]:
##############  check columns with missing values  dominant_material and color were missing ############
train1.columns[train1.isnull().any()]

Index(['dominant_material', 'dominant_color', 'product_type'], dtype='object')

In [48]:
# Applying the pre-processing functions

handle_missing_values(train1)
to_categorical(train1)

In [49]:
train1.head()

,product_id,size,brand,dominant_material,name,dominant_color,product_type,product_details,category,price,shipping,category_sub1,category_sub2
0,6937673,XL,IMARA,Polyester,IMARA Women Black Solid Top,Black,Top,"Black solid woven regular top,has a V-neck, th...",Clothing,959,1,Women,Tops
1,7441182,XL,House of Pataudi,cotton,House of Pataudi Men Black Printed Straight Kurta,Black,Straight Kurta,"Black printed straight kurta, has a mandarin c...",Clothing,799,0,Men,Kurtas
2,9245141,S,The White Willow,Cotton,The White Willow Unisex Off-White Therapedic M...,White,None,Set content: 1 pillow Colour: Off white Fill...,Home,1399,0,Unisex,Pillows
3,1314889,Onesize,Dupatta Bazaar,Chiffon,Dupatta Bazaar Pink Dupatta,Pink,Dupatta,Pink dupatta&nbsp;with crinkled effect,Clothing,349,0,Women,Dupatta
4,7705322,S,Manyavar,Silk,Manyavar Men Yellow & White Self Design Kurta ...,Yellow,Kurta with Churidar,Yellow and white self design kurta with churid...,Clothing,2999,1,Men,Kurta Sets


In [50]:
# Remove Punctuation
train1.product_details = train1.product_details.astype(str)

train1['product_details'] = train1['product_details'].apply(remove_digits)
train1['product_details'] = train1['product_details'].apply(remove_punctuation)
train1['product_details'] = train1['product_details'].apply(remove_stop_words)
train1['product_details'] = train1['product_details'].apply(to_lower)

train1['name'] = train1['name'].apply(remove_digits)
train1['name'] = train1['name'].apply(remove_punctuation)
train1['name'] = train1['name'].apply(remove_stop_words)
train1['name'] = train1['name'].apply(to_lower)

train1.head()

,product_id,size,brand,dominant_material,name,dominant_color,product_type,product_details,category,price,shipping,category_sub1,category_sub2
0,6937673,XL,IMARA,Polyester,imara women black solid top,Black,Top,black solid woven regular tophas vneck threequ...,Clothing,959,1,Women,Tops
1,7441182,XL,House of Pataudi,cotton,house pataudi men black printed straight kurta,Black,Straight Kurta,black printed straight kurta mandarin collar l...,Clothing,799,0,Men,Kurtas
2,9245141,S,The White Willow,Cotton,white willow unisex offwhite therapedic memory...,White,None,set content pillow colour white filling mem...,Home,1399,0,Unisex,Pillows
3,1314889,Onesize,Dupatta Bazaar,Chiffon,dupatta bazaar pink dupatta,Pink,Dupatta,pink dupattanbspwith crinkled effect,Clothing,349,0,Women,Dupatta
4,7705322,S,Manyavar,Silk,manyavar men yellow white self design kurta c...,Yellow,Kurta with Churidar,yellow white self design kurta churidaryellow ...,Clothing,2999,1,Men,Kurta Sets


In [51]:
######### no column now has null values  #########

train1.isnull().any()

product_id           False
size                 False
brand                False
dominant_material    False
name                 False
dominant_color       False
product_type         False
product_details      False
category             False
price                False
shipping             False
category_sub1        False
category_sub2        False
dtype: bool

In [52]:
#############################     creating a new category       ################################

bins = [0, 1000, 25501]
labels = ['less','more']
train1['lt1000'] = pd.cut(train1['price'], bins=bins, labels=labels)

In [53]:
train1.head()

,product_id,size,brand,dominant_material,name,dominant_color,product_type,product_details,category,price,shipping,category_sub1,category_sub2,lt1000
0,6937673,XL,IMARA,Polyester,imara women black solid top,Black,Top,black solid woven regular tophas vneck threequ...,Clothing,959,1,Women,Tops,less
1,7441182,XL,House of Pataudi,cotton,house pataudi men black printed straight kurta,Black,Straight Kurta,black printed straight kurta mandarin collar l...,Clothing,799,0,Men,Kurtas,less
2,9245141,S,The White Willow,Cotton,white willow unisex offwhite therapedic memory...,White,None,set content pillow colour white filling mem...,Home,1399,0,Unisex,Pillows,more
3,1314889,Onesize,Dupatta Bazaar,Chiffon,dupatta bazaar pink dupatta,Pink,Dupatta,pink dupattanbspwith crinkled effect,Clothing,349,0,Women,Dupatta,less
4,7705322,S,Manyavar,Silk,manyavar men yellow white self design kurta c...,Yellow,Kurta with Churidar,yellow white self design kurta churidaryellow ...,Clothing,2999,1,Men,Kurta Sets,more


In [54]:
#########################################################################################################################################################
#########################################################################################################################################################
#########################################################################################################################################################
##########################################                     CHANGING TO INPUT VARIABLES                   ############################################
#########################################################################################################################################################
#########################################################################################################################################################
#########################################################################################################################################################

In [55]:
# Applying Count Vectorizer to "name", this converts it into a sparse matrix 

cv = CountVectorizer(min_df=10)
X_name = cv.fit_transform(train1['name'])
X_name

<15000x460 sparse matrix of type '<class 'numpy.int64'>'
	with 107251 stored elements in Compressed Sparse Row format>

In [56]:
print(X_name.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [57]:
# Applying Count Vectorizer to "category_name", this converts it into a sparse matrix
cv = CountVectorizer()
X_category = cv.fit_transform(train1['category'])
X_sub1 = cv.fit_transform(train1['category_sub1'])
X_sub2 = cv.fit_transform(train1['category_sub2'])
X_category

<15000x6 sparse matrix of type '<class 'numpy.int64'>'
	with 15004 stored elements in Compressed Sparse Row format>

In [58]:
print(X_category.toarray())

[[0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 0 1 0]
 ...
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]]


In [59]:
# Applying TFIDF to "product_details"

tv = TfidfVectorizer(max_features=55000, ngram_range=(1, 2), stop_words='english')
X_description = tv.fit_transform(train1['product_details'])
print(X_description.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [60]:
# Applying LabelBinarizer to "brand_name"

lb = LabelBinarizer(sparse_output=True)
X_brand = lb.fit_transform(train1['brand'])
print(X_brand.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [61]:
# Applying LabelBinarizer to "size"

lb = LabelBinarizer(sparse_output=True)
X_size= lb.fit_transform(train1['size'])
print(X_size.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [62]:
# Applying LabelBinarizer to "dominant_material"

lb = LabelBinarizer(sparse_output=True)
X_material= lb.fit_transform(train1['dominant_material'])
print(X_material.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [63]:
# Applying LabelBinarizer to "dominant_color"

lb = LabelBinarizer(sparse_output=True)
X_color= lb.fit_transform(train1['dominant_color'])
print(X_color.toarray())

[[0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [64]:
# Applying LabelBinarizer to "product_type"

lb = LabelBinarizer(sparse_output=True)
X_protype= lb.fit_transform(train1['product_type'].astype('str'))
print(X_protype.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]


In [65]:
##############        Creating our final sparse matrix   #############
X_dummies = csr_matrix(pd.get_dummies(train1['shipping'], sparse=True).values)

############## Combining everything together   #################
sparse_merge = hstack((X_dummies, X_description, X_brand, X_category, X_name,X_size,X_color,X_material,X_protype)).tocsr()

In [66]:
print(X_dummies.toarray())

[[0 1]
 [1 0]
 [1 0]
 ...
 [0 1]
 [0 1]
 [1 0]]


In [67]:
print(sparse_merge[:10])

  (0, 1)	1.0
  (0, 631)	0.18426797427868996
  (0, 646)	0.1969387613102637
  (0, 1407)	0.13291435708027
  (0, 1526)	0.1918886534674721
  (0, 13023)	0.19018205888698314
  (0, 19066)	0.17328940656430292
  (0, 19111)	0.4264151099736126
  (0, 21100)	0.05652818524240653
  (0, 21114)	0.2832183210048103
  (0, 22329)	0.08533297800444409
  (0, 22489)	0.19179712289066336
  (0, 23903)	0.0922384992926741
  (0, 23921)	0.09424063639220509
  (0, 24197)	0.3966280875276992
  (0, 24199)	0.4264151099736126
  (0, 24904)	0.17927468068743624
  (0, 24935)	0.20940467055285528
  (0, 25957)	0.1073080246636062
  (0, 26061)	0.2307186735018484
  (0, 26657)	1.0
  (0, 26898)	1.0
  (0, 26945)	1.0
  (0, 27096)	1.0
  (0, 27293)	1.0
  :	:
  (9, 21591)	0.14490096855773388
  (9, 21778)	0.07587990952985262
  (9, 21921)	0.19056345037574712
  (9, 22329)	0.07909784386859955
  (9, 22443)	0.15884958864653892
  (9, 22772)	0.13697049026680239
  (9, 22788)	0.0809177101024286
  (9, 22816)	0.12573602976872023
  (9, 25538)	0.234990274

In [68]:
# Create log price variable (Transformation)
y = np.log1p(train1['price'])
y.head()

0    6.866933
1    6.684612
2    7.244228
3    5.857933
4    8.006368
Name: price, dtype: float64

In [69]:
len(y)

15000

In [70]:
train_x = train1[:12000]
train_y = y[:12000]

test_x = train1[12000:]
test_y = y[12000:]
print(len(train_x))
print(len(train_y))
print(len(test_x))
print(len(test_y))

12000
12000
3000
3000


In [71]:
X_train_sparse = sparse_merge[:len(train_x)]
X_test = sparse_merge[len(train_x):]

In [72]:
def rmsle(y, y0):
    assert len(y) == len(y0)
    return np.sqrt(np.mean(np.power(np.log1p(y)-np.log1p(y0), 2)))

In [42]:
##############################################original#######################################################################################################################################################################################################################################

kf = KFold(10,shuffle=False)
for train_index, valid_index in kf.split(X_train_sparse):
    X_train, y_train = X_train_sparse[train_index], y[train_index]
    X_valid, y_valid = X_train_sparse[valid_index], y[valid_index]
    d_train = lgb.Dataset(X_train, label=y_train)
    params = {}
    #params['learning_rate'] = 0.003
    params['boosting_type'] = 'gbdt'
    params['objective'] = 'regression'
    params['metric'] = 'rmse'

    clf = lgb.train(params, d_train, 100)
   
    lgbm_pred=clf.predict(X_valid)
    

    start_time = time.time()
    print('[{}] LGBM completed.'.format(time.time() - start_time))
    print("FOR train_index: ",train_index,"and valid_index: ",valid_index)
    print("LGBM rmsle: "+str(rmsle(np.expm1(y_valid), np.expm1(lgbm_pred))))
    print()

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.038088 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 60012
[LightGBM] [Info] Number of data points in the train set: 10800, number of used features: 1922
[LightGBM] [Info] Start training from score 7.824046
[0.0] LGBM completed.
FOR train_index:  [ 1200  1201  1202 ... 11997 11998 11999] and valid_index:  [   0    1    2 ... 1197 1198 1199]
LGBM rmsle: 0.6586958504667046

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.038571 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 59794
[LightGBM] [Info] Number of data points in the train set: 10800, number of used features: 1931
[LightGBM] [Info] Start training from score 7.824046
[0.0] LGBM completed.
FOR train_index:  [    0     1     2 ... 11997 11998 11999] and valid_index:  [1200 1201 1202 ... 2397 2398 2399]


In [73]:
kf = KFold(10,shuffle=False)
temp=0
for train_index, valid_index in kf.split(X_train_sparse):
    X_train, y_train = X_train_sparse[train_index], y[train_index]
    X_valid, y_valid = X_train_sparse[valid_index], y[valid_index]
    #d_train = lgb.Dataset(X_train, label=y_train)
    params = {}
    params['learning_rate'] = 0.1
    params['boosting_type'] = 'gbdt'
    params['objective'] = 'quantile'
    params['metric'] = 'quantile'
    quantiles = [.1, .5, .9]
   
    lgbm_pred=np.zeros((len(valid_index),3))
    pred=np.zeros((3000,3))
  
    for i in range(len(quantiles)):
        print('prediction of quantile', quantiles[i])
        lgb = LGBMRegressor(alpha=quantiles[i], **params)
        model = lgb.fit(X_train, y_train)
        lgbm_pred[:,i] = model.predict(X_valid)
        pred[:,i]=model.predict(X_test)
    
    #print(lgbm_pred)
    #print(pred)
    print(np.expm1(pred))
    if(temp==1):
        final=np.expm1(pred)
    temp=temp+1

    ans=rmsle(np.expm1(y_valid), np.expm1(lgbm_pred[:,1]))
    start_time = time.time()
    print('[{}] LGBM completed.'.format(time.time() - start_time))
    print("FOR train_index: ",train_index,"and valid_index: ",valid_index)
    print("LGBM rmsle: "+str(ans))
    print()

prediction of quantile 0.1
prediction of quantile 0.5
prediction of quantile 0.9
[[ 989.23052135 1352.81973853 2285.54013995]
 [ 618.27059077  883.73504649 1735.53109143]
 [ 391.81532225  471.84801035 1421.91941281]
 ...
 [ 620.7218867   985.19261559 1822.59539521]
 [ 547.69383879  729.1278889  1552.59824935]
 [ 711.95567449  846.29918664 1037.89240652]]
[0.0] LGBM completed.
FOR train_index:  [ 1200  1201  1202 ... 11997 11998 11999] and valid_index:  [   0    1    2 ... 1197 1198 1199]
LGBM rmsle: 0.35256028566326997

prediction of quantile 0.1
prediction of quantile 0.5
prediction of quantile 0.9
[[ 894.19998455 1335.38466516 2286.20646915]
 [ 642.74144708  908.2522971  1550.64082774]
 [ 435.78273697  557.68785179 1499.51688331]
 ...
 [ 618.79488814  942.76531963 1643.5569318 ]
 [ 547.41538867  733.80025054 1456.37351039]
 [ 750.9115762   857.18980423 1073.88183897]]
[0.0] LGBM completed.
FOR train_index:  [    0     1     2 ... 11997 11998 11999] and valid_index:  [1200 1201 1202 .

In [74]:
print(final[0])
print(final[1])
print(final[2])
print(final[3])
print(final[4])
print(final[5])
print(final[58])
print(final[59])
print(final[60])
print(final[61])
print(final[62])
print(final[63])
print(final[64])

[ 894.19998455 1335.38466516 2286.20646915]
[ 642.74144708  908.2522971  1550.64082774]
[ 435.78273697  557.68785179 1499.51688331]
[ 784.82091715 1238.49291963 2285.46171075]
[ 497.78050039  617.1644014  1491.82544236]
[ 596.91499708  958.74226308 1595.80435086]
[ 495.9495218   782.91958509 1277.98198812]
[1070.55120731 2304.37506853 5117.81481083]
[ 897.47937007 2466.90686789 4451.95729234]
[ 623.32102431  950.54747572 1671.53532658]
[1649.45992387 2574.41735481 3755.71086155]
[1381.54276236 2601.74531546 3369.16865846]
[ 591.00300187  947.28597183 1809.99841787]


In [80]:
print(final[16][0])
print(final[28][0])
print(final[34][0])
print(final[17][0])
print(final[20][0])
print(final[24][0])
print(final[43][0])
print(final[46][0])
print(final[33][0])
print(final[15][0])
print(final[29][0])
print(final[30][0])
print(final[32][0])
print(final[35][0])
print(final[13][0])
print(final[14][0])
print(final[21][0])
print(final[26][0])
print(final[48][0])
print(final[49][0])
print(final[25][0])
print(final[19][0])
print(final[18][0])
print(final[22][0])
print(final[23][0])
print(final[27][0])
print(final[31][0])
print(final[47][0])

934.9254313685782
1119.170179329013
436.35043741529995
550.7256784612986
1072.9624515179446
583.2567071362687
609.2127863227583
963.2792533826488
684.7641687758537
702.7129819856549
687.4998762039954
752.6135742570108
1045.7321985431124
648.3355726865311
586.9900650400284
590.6850860657028
627.0495035496718
1010.35960416753
864.0751485956686
737.2491738192988
750.6112509281154
1087.5217546885233
620.7314082937229
565.6746480829415
1475.1010291506764
535.5993356082816
907.9788086569016
625.9592935572931


In [81]:
print(final[16][1])
print(final[28][1])
print(final[34][1])
print(final[17][1])
print(final[20][1])
print(final[24][1])
print(final[43][1])
print(final[46][1])
print(final[33][1])
print(final[15][1])
print(final[29][1])
print(final[30][1])
print(final[32][1])
print(final[35][1])
print(final[13][1])
print(final[14][1])
print(final[21][1])
print(final[26][1])
print(final[48][1])
print(final[49][1])
print(final[25][1])
print(final[19][1])
print(final[18][1])
print(final[22][1])
print(final[23][1])
print(final[27][1])
print(final[31][1])
print(final[47][1])

1295.8973034711828
1582.1568347668888
646.1439781308179
647.3983230688003
1639.7700960910522
2342.5349319648
763.7313449351021
1654.9656357056667
2182.7363672416714
921.6950615021955
1213.7648826644377
1187.8905468244925
2623.278722398886
2511.420628187367
1872.6617415948926
787.9298858294254
856.4502234183344
2140.297824592159
992.321985474588
1110.249179213107
877.811396498879
1997.9589378591936
823.1047360406981
650.6198120513953
3120.434173737805
813.5553640980672
1338.1289886652337
797.8749175938227


In [82]:
print(final[16][2])
print(final[28][2])
print(final[34][2])
print(final[17][2])
print(final[20][2])
print(final[24][2])
print(final[43][2])
print(final[46][2])
print(final[33][2])
print(final[15][2])
print(final[29][2])
print(final[30][2])
print(final[32][2])
print(final[35][2])
print(final[13][2])
print(final[14][2])
print(final[21][2])
print(final[26][2])
print(final[48][2])
print(final[49][2])
print(final[25][2])
print(final[19][2])
print(final[18][2])
print(final[22][2])
print(final[23][2])
print(final[27][2])
print(final[31][2])
print(final[47][2])

1816.1162444603829
4969.152389736685
1122.1578150020844
1324.5511664706887
3421.1465666190256
4758.146385146272
1727.6572155399529
2529.902661297887
4578.968210655175
1647.463341584755
1807.247100963532
2783.3937960577196
3817.9470965362334
6583.875575201719
5189.9704740315565
1367.0925268935362
1811.3609738326088
4107.546835978464
1815.0362936851345
1630.350176693883
1415.5267379605243
3943.832141215754
1459.3076260570674
1885.365440823731
3811.6505042950103
2254.9623039076405
2024.9259649295514
1246.0136926642983


In [76]:
print(np.expm1(test_y[12000]))
print(np.expm1(test_y[12001]))
print(np.expm1(test_y[12002]))
print(np.expm1(test_y[12003]))
print(np.expm1(test_y[12004]))
print(np.expm1(test_y[12005]))
print(np.expm1(test_y[12058]))
print(np.expm1(test_y[12059]))
print(np.expm1(test_y[12060]))
print(np.expm1(test_y[12061]))
print(np.expm1(test_y[12062]))
print(np.expm1(test_y[12063]))
print(np.expm1(test_y[12064]))

1592.0000000000005
1299.0
648.9999999999999
998.9999999999998
764.0
808.9999999999998
1329.0000000000002
2698.999999999999
3498.9999999999995
1598.9999999999998
2798.9999999999995
3498.9999999999995
998.9999999999998


In [77]:
print(np.expm1(test_y[12016]))
print(np.expm1(test_y[12028]))
print(np.expm1(test_y[12034]))
print(np.expm1(test_y[12017]))
print(np.expm1(test_y[12020]))
print(np.expm1(test_y[12024]))
print(np.expm1(test_y[12043]))
print(np.expm1(test_y[12046]))
print(np.expm1(test_y[12033]))
print(np.expm1(test_y[12015]))
print(np.expm1(test_y[12029]))
print(np.expm1(test_y[12030]))
print(np.expm1(test_y[12032]))
print(np.expm1(test_y[12035]))
print(np.expm1(test_y[12013]))
print(np.expm1(test_y[12014]))
print(np.expm1(test_y[12021]))
print(np.expm1(test_y[12026]))
print(np.expm1(test_y[12048]))
print(np.expm1(test_y[12049]))
print(np.expm1(test_y[12025]))
print(np.expm1(test_y[12019]))
print(np.expm1(test_y[12018]))
print(np.expm1(test_y[12022]))
print(np.expm1(test_y[12023]))
print(np.expm1(test_y[12027]))
print(np.expm1(test_y[12031]))
print(np.expm1(test_y[12047]))

1499.0
1319.0
599.0
629.0000000000001
1795.0000000000007
1485.0000000000002
755.9999999999998
2998.9999999999977
2998.9999999999977
782.9999999999998
1998.9999999999998
696.0000000000001
1838.9999999999993
3499.9999999999977
3359.999999999999
1118.9999999999998
523.9999999999999
3199.9999999999986
808.9999999999998
1039.0
599.0
5599.000000000005
1695.0000000000002
638.9999999999999
2399.0000000000005
1128.9999999999995
2009.0000000000005
719.0000000000001


In [43]:
kf = KFold(10,shuffle=False)
for train_index, valid_index in kf.split(X_train_sparse):
    start_time = time.time()
    model = Ridge(solver = "sag", fit_intercept=False)
    print("Fitting Ridge Model")
    model.fit(X_train, y_train)
    preds_valid = model.predict(X_valid)
    print('[{}] Ridge completed.'.format(time.time() - start_time))
    print("FOR train_index: ",train_index,"and valid_index: ",valid_index)
    print("Ridge rmsle: "+str(rmsle(np.expm1(y_valid), np.expm1(preds_valid))))

Fitting Ridge Model
[0.8896963596343994] Ridge completed.
FOR train_index:  [ 1200  1201  1202 ... 11997 11998 11999] and valid_index:  [   0    1    2 ... 1197 1198 1199]
Ridge rmsle: 0.3831553444294702
Fitting Ridge Model
[0.36742615699768066] Ridge completed.
FOR train_index:  [    0     1     2 ... 11997 11998 11999] and valid_index:  [1200 1201 1202 ... 2397 2398 2399]
Ridge rmsle: 0.383893960074313
Fitting Ridge Model
[0.44581079483032227] Ridge completed.
FOR train_index:  [    0     1     2 ... 11997 11998 11999] and valid_index:  [2400 2401 2402 ... 3597 3598 3599]
Ridge rmsle: 0.38272745562527805
Fitting Ridge Model
[0.4228975772857666] Ridge completed.
FOR train_index:  [    0     1     2 ... 11997 11998 11999] and valid_index:  [3600 3601 3602 ... 4797 4798 4799]
Ridge rmsle: 0.38324390581544915
Fitting Ridge Model
[0.3869929313659668] Ridge completed.
FOR train_index:  [    0     1     2 ... 11997 11998 11999] and valid_index:  [4800 4801 4802 ... 5997 5998 5999]
Ridge rms

In [48]:
traine = {'name':"",'shipping':"",'product_details':"",'brand':"",'category':"",'size':"",'color':"",'material':"",'product_type':""}
traine['name']="Global Desi Mustard Printed Tunic"
traine['shipping']="1"
traine['product_details']="Taking the bright colours of summer and nature, Global Desi inspires the Women of style and substance to experiment with a range of colours from sophisticated black to subtle pastels and vibrant shades. So match the shade to your personality! Mix and match from a variety of colours and prints for a feminine look, or a modern twist."
traine['brand']="Global Desi"
traine['category']="Clothing"
traine['size']="XXL"
traine['color']="Mustard"
traine['material']="viscose"
traine['product_type']="Tunic"

from csv import writer
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)
append_list_as_row('data.csv',traine.keys())
append_list_as_row('data.csv',traine.values())
train12= pd.read_csv('data.csv')
#train12.drop(['images','complete_the_look'],axis=1,inplace=True)
train12.head()
#len(train.columns)

,name,shipping,product_details,brand,category,size,color,material,product_type
0,Global Desi Mustard Printed Tunic,1,Taking the bright colours of summer and nature...,Global Desi,Clothing,XXL,Mustard,viscose,Tunic
1,name,shipping,product_details,brand,category,size,color,material,product_type
2,Global Desi Mustard Printed Tunic,1,Taking the bright colours of summer and nature...,Global Desi,Clothing,XXL,Mustard,viscose,Tunic


In [87]:
to_categorical(train12)
train12.product_details = train12.product_details.astype(str)

train12['product_details'] = train12['product_details'].apply(remove_digits)
train12['product_details'] = train12['product_details'].apply(remove_punctuation)
train12['product_details'] = train12['product_details'].apply(remove_stop_words)
train12['product_details'] = train12['product_details'].apply(to_lower)

from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
train12['product_details'] = train12['product_details'].apply(porter.stem)

train12['name'] = train12['name'].apply(remove_digits)
train12['name'] = train12['name'].apply(remove_punctuation)
train12['name'] = train12['name'].apply(remove_stop_words)
train12['name'] = train12['name'].apply(to_lower)

cv = CountVectorizer()
X_name = cv.fit_transform(train12['name'])

cv = CountVectorizer()
X_category = cv.fit_transform(train12['category'])

tv = TfidfVectorizer(max_features=55000, ngram_range=(1, 2), stop_words='english')
X_description = tv.fit_transform(train12['product_details'])

lb = LabelBinarizer(sparse_output=True)
X_brand = lb.fit_transform(train12['brand'])
X_size= lb.fit_transform(train12['size'])
X_material= lb.fit_transform(train12['material'])
X_color= lb.fit_transform(train12['color'])
X_protype= lb.fit_transform(train12['product_type'])

X_dummies = csr_matrix(pd.get_dummies(train12['shipping'], sparse=True).values)

sparse_merge = hstack((X_dummies, X_description, X_brand, X_category, X_name,X_size,X_color,X_material,X_protype)).tocsr()
preds = clf.predict(sparse_merge)
arr= np.expm1(preds)
print(arr)

LightGBMError: The number of features in data (72) is not the same as it was in training data (27860).
You can set ``predict_disable_shape_check=true`` to discard this error, but please be aware what you are doing.